In [1]:
from itertools import chain
from pathlib import Path
from typing import Hashable

import numpy as np
import tensorflow as tf


In [15]:
plaid = np.load('../out/plaid.npz', allow_pickle=True)
plaid = plaid['arr_0']
print(len(plaid[0][0]))

62


In [21]:
def load_files(data_set, nested=False):
    """Loads requested system call data set from disk

    Parameters
    ----------
    data_set : {"adfa", "plaid"}
        The data set to be returned.
    nested : bool
        Return attack sequences nested by application. Default False returns a flat list.

    Returns
    -------
    attack_sequences : List[List[str]] or List[List[List[str]]]
        List of attack system call sequences. When nested=False each element is an attack sequence represented as a list
        of strings. If nested=True each element is a list of all attack sequences belonging to a single application.
    base_sequences : List[List[str]]
        List of baseline system call sequences.

    """
    if data_set not in ["adfa", "plaid"]:
        raise ValueError("data_set must be on of (adfa, plaid)")

    def get_seq(files):
        ret = []
        for f in files:
            with open(f) as file:
                seq = file.read().strip().split(" ")
                if 4495 >= len(seq) >= 8:
                    ret.append(seq)
        return ret

    if data_set == "plaid":
        attack_files = sorted(list(Path("../data/PLAID/attack").rglob("*.txt")))
        baseline_files = Path("../data/PLAID/baseline").rglob("*.txt")
    else:
        root_path = Path("../data/ADFA_decoded_i386/")
        attack_files = sorted(list((root_path / "Attack_Data_Master").rglob("*.txt")))
        baseline_files = list((root_path / "Validation_Data_Master").rglob("*.txt"))
        baseline_files.extend((root_path / "Training_Data_Master").rglob("*.txt"))

    if nested:
        attack_sequences = []
        folders = set([x.parent for x in attack_files])
        for folder in folders:
            tmp = [x for x in attack_files if x.parent == folder]
            attack_sequences.append(get_seq(tmp))
    else:
        attack_sequences = get_seq(attack_files)
    base_sequences = get_seq(baseline_files)
    return attack_sequences, base_sequences



In [19]:
class Encoder:
    """Converts data to a dense integer encoding

    Attributes:
        file_path: location to save/load syscall map
        syscall_map: mapping from item to encoded value
    """

    file_path = Path()
    syscall_map: dict = dict()

    def __init__(self, file_path: str) -> None:
        self.file_path = Path(file_path)
        if self.file_path.exists():
            self.syscall_map = np.load(self.file_path, allow_pickle=True).item()

    def encode(self, syscall: Hashable) -> int:
        """Encodes an individual item

        Unique items are sequentially encoded (ie first item -> 0 next unique item -> 1). The mapping dict is updated
        with new encodings as necessary and immediately written to disk.

        Args:
            syscall: item to encode

        Returns:
            integer encoding of syscall
        """
        if syscall in self.syscall_map:
            return self.syscall_map[syscall]
        syscall_enc = len(self.syscall_map) + 1
        self.syscall_map[syscall] = syscall_enc
        np.save(self.file_path, self.syscall_map)

        return syscall_enc


In [22]:
atk_files, normal_files = load_files("plaid")

In [26]:
print(len(atk_files))

1145


In [27]:
print(len(normal_files))

38178


In [29]:
print(len(plaid[0]) + len(plaid[1]) + len(plaid[2]))
print(len(plaid[3]))

38178
1145


In [30]:
atk = plaid[3]
normal = plaid[0] + plaid[1] + plaid[2]

In [31]:
print(normal[0])

[ 18.   7.  30.   7.   7.  30.   7.   7.  30.   7.  46.   8.   9.  10.
  10. 138. 172.  93.   7.  10. 138.  13. 172.  93.  93.  14.  93.  93.
   7. 173.  14.  14.  14.  16. 174. 175.  43.  19. 176.   9.   9.  45.
 177. 177. 176. 178.  19.  19.  19.  19.  19.  19. 179.   7. 179.  30.
  39.  47. 168.  30.   7. 128.]


In [1]:
max_num = 0
for i in range(len(normal)):
    for j in range(len(normal[i])):
        max_num = max(max_num, normal[i][j])
print(max_num)
for i in range(len(atk)):
    for j in range(len(atk[i])):
        max_num = max(max_num, atk[i][j])
print(max_num)
# print(max_len)

NameError: name 'normal' is not defined